In [5]:
import geopandas as gpd
from pathlib import Path
import matplotlib.pyplot as plt
import contextily as cx
import fiona

# Potentiel solaire package
from potentiel_solaire.constants import DATA_FOLDER

In [6]:
# Executer ci dessous ci besoin pour récupérer les données
# !extract-sample-data
# Et pour sauver une version markdown des notebooks, utiliser
# jupyter nbconvert donnees_par_ecole.ipynb --to markdown --output-dir=exports/

## BDTOPO

### Extraction 

In [7]:
GPKG = DATA_FOLDER / "BDTOPO_3-4_TOUSTHEMES_GPKG_LAMB93_D093_2024-12-15/BDTOPO/1_DONNEES_LIVRAISON_2024-12-00134/BDT_3-4_GPKG_LAMB93_D093-ED2024-12-15/BDT_3-4_GPKG_LAMB93_D093-ED2024-12-15.gpkg"
communes = gpd.read_file(GPKG, layer="commune")

# on crée la var qui nous servira de filtre sur les autres db
saint_denis = communes[communes.code_insee == "93066"].to_crs(4326)

In [ ]:
# Takes 50s on my laptop 

# 1. Bâtiments
bats = gpd.read_file(GPKG, layer="batiment", mask=saint_denis)
bats = bats.to_crs(4326)
#TODO: confirm use of intersect (instead of within) with Luc or else
bats_st_denis = gpd.sjoin(bats, saint_denis, how='inner', predicate='intersects')

# 2. Zones d'éducation
bdedu = gpd.read_file(GPKG, layer="zone_d_activite_ou_d_interet", mask=saint_denis)
bdedu = bdedu.to_crs(4326)
bdedu = bdedu[bdedu["categorie"] == "Science et enseignement"]
bdedu_st_denis = gpd.sjoin(bdedu, saint_denis, how='inner', predicate='intersects')

### Visualisation

In [ ]:
# 1. Plot des bâtiments
fig1, ax1 = plt.subplots(figsize=(15, 15))
saint_denis.plot(ax=ax1, facecolor='none', edgecolor='red', linewidth=2, alpha=0.5, label='Saint-Denis')
bats_st_denis.plot(ax=ax1, color='blue', alpha=0.5, label='Bâtiments')
cx.add_basemap(ax1, crs=bats_st_denis.crs, zoom=14)
ax1.set_title(f"Bâtiments à Saint-Denis\n{len(bats_st_denis)} bâtiments", pad=20)
ax1.legend()
plt.show()

In [ ]:
# 2. Plot des zones d'éducation
fig2, ax2 = plt.subplots(figsize=(15, 15))
saint_denis.plot(ax=ax2, facecolor='none', edgecolor='red', linewidth=2, alpha=0.5, label='Saint-Denis')
bdedu_st_denis.plot(ax=ax2, facecolor='blue', alpha=0.3, edgecolor='blue', linewidth=2, label='Zones éducatives')
cx.add_basemap(ax2, crs=bdedu_st_denis.crs, zoom=14)
ax2.set_title(f"Zones d'éducation à Saint-Denis\n{len(bdedu_st_denis)} zones", pad=20)
ax2.legend()
plt.show()

## Annuaire éducation

### Extraction 

In [12]:
# Takes 12s on my laptop 

annuaire = gpd.read_file(DATA_FOLDER / 'fr-en-annuaire-education.geojson')
annuaire = annuaire.to_crs(4326)
annuaire_st_denis = gpd.sjoin(annuaire, saint_denis, how='inner', predicate='within')

### Visualisation

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15))
saint_denis.plot(ax=ax, facecolor='none', edgecolor='red', linewidth=2, alpha=0.5, label='Saint-Denis')
annuaire_st_denis.plot(ax=ax, color='blue', markersize=100, label='Ecoles')

cx.add_basemap(ax, crs=annuaire_st_denis.crs, zoom=14)
ax.set_title(f"Annuaire des Ecoles à Saint-Denis\n{len(annuaire_st_denis)} écoles", pad=20)
ax.legend()
plt.show()

## Plan Cadastral Informatisé (PCI)

### Extraction 

In [14]:
# Takes 10s on my laptop

pci = gpd.read_file(DATA_FOLDER / "PARCELLAIRE-EXPRESS_1-1__SHP_LAMB93_D093_2024-10-01/PARCELLAIRE-EXPRESS/1_DONNEES_LIVRAISON_2024-11-00210/PEPCI_1-1_SHP_LAMB93_D093/BATIMENT.SHP", mask=saint_denis)

pci = pci.to_crs(4326)
pci_st_denis = gpd.sjoin(pci, saint_denis, how='inner', predicate='intersects')

### Visualisation 

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15))
saint_denis.plot(ax=ax, facecolor='none', edgecolor='red', linewidth=2, alpha=0.5, label='Saint-Denis')
pci_st_denis.plot(ax=ax, color='blue', label='Ecoles')

cx.add_basemap(ax, crs=pci_st_denis.crs, zoom=14)
ax.set_title(f"PCI Saint Denis\n{len(pci_st_denis)}", pad=20)
ax.legend()
plt.show()

## Visualisation des écoles selon la bd topo vs selon l'annuaire 

In [ ]:

fig, ax = plt.subplots(figsize=(15, 15))

# 1. la limite de Saint-Denis (en fond)
saint_denis.plot(
    ax=ax, 
    facecolor='none', 
    edgecolor='red', 
    linewidth=2, 
    alpha=0.5, 
    label='Saint-Denis'
)

# 2. les zones d'éducation de la BD TOPO (en dessous)
bdedu_st_denis.plot(
    ax=ax, 
    facecolor='lightblue', 
    alpha=0.3, 
    edgecolor='blue', 
    linewidth=2, 
    label='Zones éducatives (BD TOPO)'
)

# 3. les points de l'annuaire (au-dessus)
annuaire_st_denis.plot(
    ax=ax,
    color='red',
    markersize=100,
    label='Écoles (Annuaire)'
)

cx.add_basemap(ax, crs=bdedu_st_denis.crs, zoom=14)
ax.set_title(f"Zones éducatives et écoles à Saint-Denis\n"
             f"{len(bdedu_st_denis)} zones, {len(annuaire_st_denis)} écoles", 
             pad=20)
ax.legend()

plt.show()


## Ajout du Potentiel gisement Solaire brut au bati

In [ ]:
GPKG = DATA_FOLDER / "potentiel-gisement-solaire-brut-au-bati.gpkg"
gsbab = gpd.read_file(GPKG, mask=saint_denis)
gsbab.head()

In [18]:
gsbab = gpd.sjoin(gsbab, saint_denis, how='inner', predicate='intersects')

In [ ]:

fig, ax = plt.subplots(figsize=(15, 15))

# 1. la limite de Saint-Denis (en fond)
saint_denis.plot(
    ax=ax, 
    facecolor='none', 
    edgecolor='red', 
    linewidth=2, 
    alpha=0.5, 
    label='Saint-Denis'
)

# 2. le potentiel brut au bati
gsbab.plot(
    ax=ax, 
    facecolor='yellow', 
    alpha=0.9, 
    edgecolor='orange', 
    linewidth=2, 
    label='Potentiel Solaire Brut au bati'
)

# 3. les points de l'annuaire (au-dessus)
annuaire_st_denis.plot(
    ax=ax,
    color='green',
    markersize=100,
    label='Écoles (Annuaire)'
)

cx.add_basemap(ax, crs=bdedu_st_denis.crs, zoom=14)
ax.set_title(f"Zones éducatives et écoles à Saint-Denis\n"
             f"{len(bdedu_st_denis)} zones, {len(annuaire_st_denis)} écoles", 
             pad=20)
ax.legend()

plt.show()


## Utilisation de l'autre base de données - Potentiel solaire des toitures

Provenant de https://data-iau-idf.opendata.arcgis.com/datasets/iau-idf::le-potentiel-solaire-des-toitures/explore?location=48.947152%2C2.406292%2C12.00

In [ ]:
GPKG = DATA_FOLDER / "potentiel-solaire.geojson"
gspsdt_total = gpd.read_file(GPKG, mask=saint_denis)
print(len(gspsdt_total))
gspsdt_total=  gspsdt_total.to_crs(4326)
gspsdt_total.head()

In [ ]:
gspsdt_total

In [ ]:
gspsdt = gspsdt_total[gspsdt_total.insee == 93066]
gspsdt = gpd.sjoin(gspsdt_total, saint_denis, how='inner', predicate='intersects')

gspsdt.head(3)

In [ ]:
gspsdt_total.plot()

In [ ]:
saint_denis.plot()

In [ ]:

fig, ax = plt.subplots(figsize=(15, 15))

# 1. la limite de Saint-Denis (en fond)
saint_denis.plot(
    ax=ax, 
    facecolor='none', 
    edgecolor='red', 
    linewidth=2, 
    alpha=0.5, 
    label='Saint-Denis'
)

# 2. Potentiel Solaire des toitures
gspsdt.plot(
    ax=ax, 
    facecolor='yellow', 
    alpha=0.9, 
    edgecolor='orange', 
    linewidth=2, 
    label='Potentiel Solaire des toitures'
)

# 3. les points de l'annuaire (au-dessus)
annuaire_st_denis.plot(
    ax=ax,
    color='green',
    markersize=100,
    label='Écoles (Annuaire)'
)

cx.add_basemap(ax, crs=saint_denis.crs, zoom=14)
ax.set_title(f"Potentiel scolaire et écoles à Saint-Denis\n"
             f"{len(gspsdt)} zones, {len(annuaire_st_denis)} écoles", 
             pad=20)
ax.legend()

plt.show()


## Nettoyer le dataset 

### Retirer les écoles de l'annuaire qui 'trop loin' des zones d'éducation

In [ ]:
ecoles_avec_zone = gpd.sjoin_nearest(
    annuaire_st_denis,
    bdedu_st_denis,
    how='inner',
    lsuffix='_annuaire',
    rsuffix='_zone',
    distance_col="distances",
    exclusive=True
)
print(len(ecoles_avec_zone))

In [ ]:
seuil_recol = ecoles_avec_zone.distances.quantile(q=0.7)
print(seuil_recol)
ecoles_avec_zone.distances.hist(bins=50,figsize=(15,5))

In [ ]:
seuil_de_match = 0.0002 # cf histogramme ci dessus.
ecoles_sans_zone = ecoles_avec_zone[ecoles_avec_zone.distances >= seuil_de_match ]
ecoles_avec_zone = ecoles_avec_zone[ecoles_avec_zone.distances < seuil_de_match ]
ecoles_avec_zone["UID"] = ecoles_avec_zone.identifiant_de_l_etablissement
print(len(ecoles_sans_zone),len(ecoles_avec_zone))
ecoles_avec_zone.head()

In [ ]:
identifiers = gpd.GeoDataFrame(ecoles_avec_zone[["identifiant_de_l_etablissement","nom_etablissement","cleabs_left","adresse_1","code_postal_left","geometry"]].reset_index(drop=True), geometry="geometry")
identifiers["UID"] = identifiers.identifiant_de_l_etablissement
identifiers.head(3)

## Retirer les zones éducatives de la bd topo qui ne sont pas dans l'annuaire

In [40]:
def checkUID(gdf):
    assert "UID" in list(gdf.columns)

    

In [41]:
# On utilise cleabs_left qui est la colonne d'identifiant dans bdedu_st_denis
zones_avec_ecole = bdedu_st_denis[bdedu_st_denis.cleabs_left.isin(ecoles_avec_zone['cleabs_left'])]

### Filtrer les bâtiments de la bd topo qui sont dans une zone éducative

In [ ]:
batiments_ecoles = gpd.sjoin(
    bats_st_denis,
    zones_avec_ecole,
    how='inner',
    predicate='intersects',
    lsuffix='_bat',
    rsuffix='_zone'
)
batiments_ecoles.head(2)

#checkUID(batiments_ecoles)

### Filtrer les batiments du plan cadastral qui sont dans une zone éducative

In [ ]:
cadastre_ecoles = gpd.sjoin(
    pci_st_denis,
    zones_avec_ecole,
    how='inner',
    predicate='intersects',
    lsuffix='_cadastre',
    rsuffix='_zone'
)
cadastre_ecoles.head(2)

### Filtrer les batiments avec potentiel solaire au bati qui sont dans une zone éducative

In [ ]:
gspsdt_ecoles = gpd.sjoin(
    gspsdt,
    zones_avec_ecole,
    how='inner',
    predicate='intersects',
    lsuffix='_gspsdt',
    rsuffix='_zone'
)
gspsdt_ecoles.head(2)

### Filtrer les batiments avec potentiel solaire au bati qui sont dans une zone éducative

In [ ]:
gsbab_ecoles = gpd.sjoin(
    gsbab,
    zones_avec_ecole,
    how='inner',
    predicate='intersects',
    lsuffix='_gsbab',
    rsuffix='_zone'
)
gsbab_ecoles.head(2)

In [ ]:
print("\n=== Statistiques avant/après nettoyage ===")
print(f"Annuaire: {len(annuaire_st_denis)} -> {len(ecoles_avec_zone)}")
print(f"Zones éducatives: {len(bdedu_st_denis)} -> {len(zones_avec_ecole)}")
print(f"Bâtiments: {len(bats_st_denis)} -> {len(batiments_ecoles)}")
print(f"Cadastre: {len(pci_st_denis)} -> {len(cadastre_ecoles)}")
print(f"Bâtiments avec potentiel scolaire au bati: {len(gsbab)} -> {len(gsbab_ecoles)}")
print(f"Bâtiments avec potentiel scolaire des toitures: {len(gspsdt_total)}(total 93) -> {len(gspsdt)} -> {len(gspsdt_ecoles)}")

## Geopackage final

### Sauvegarder en Geopackage


In [47]:
output_gpkg = DATA_FOLDER / "saint_denis_reference_data.gpkg"

# Ecoles d'apres l'annuaire
ecoles_avec_zone.to_file(output_gpkg, layer='annuaire_education', driver="GPKG")
ecoles_sans_zone.to_file(output_gpkg, layer='annuaire_education_sans_zone', driver="GPKG")
# Données geographiques écoles
zones_avec_ecole.to_file(output_gpkg, layer='bdtopo_education', driver="GPKG")
batiments_ecoles.to_file(output_gpkg, layer='bdtopo_batiment', driver="GPKG")
cadastre_ecoles.to_file(output_gpkg, layer='cadastre_parcellaire', driver="GPKG")
gsbab_ecoles.to_file(output_gpkg, layer='potentielsolaire_bati', driver="GPKG")
gspsdt_ecoles.to_file(output_gpkg, layer='potentielsolaire_toitures', driver="GPKG")
saint_denis.to_file(output_gpkg, layer='perimetre_st_denis', driver="GPKG")
# Rajout des identifiants
identifiers.to_file(output_gpkg, layer='identifiers', driver="GPKG")

## Visualiser le geopackage créé 

### Verification des layers

In [ ]:
layers = fiona.listlayers(output_gpkg)

for layer in layers:
    gdf = gpd.read_file(output_gpkg, layer=layer)
    print(f"\n=== Couche: {layer} ===")
    print(f"Nombre d'objets: {len(gdf)}")
    print(f"Colonnes disponibles: {gdf.columns.tolist()}")
    print(f"Type de géométrie: {gdf.geometry.geom_type.unique()}")
    print(f"CRS: {gdf.crs}") # bien vérifier que tout est en EPSG:4326 / WGS84

### Verification visuelle

In [ ]:

fig, ax = plt.subplots(figsize=(15, 15))

saint_denis.plot(
    ax=ax, 
    facecolor='none', 
    edgecolor='red', 
    linewidth=2, 
    alpha=0.5, 
    label='Saint-Denis'
)

# les zones d'éducation de la BD TOPO
batiments_ecoles.plot(
    ax=ax, 
    facecolor='lightblue', 
    alpha=0.3, 
    edgecolor='blue', 
    linewidth=2, 
    label='Zones éducatives (BD TOPO)'
)

# les zones d'éducation de la BD TOPO
cadastre_ecoles.plot(
    ax=ax, 
    facecolor='green', 
    alpha=0.3, 
    edgecolor='green', 
    linewidth=2, 
    label='Batiments d écoles cf cadastre(BD TOPO)'
)


# les zones d'éducation de la BD TOPO
batiments_ecoles.plot(
    ax=ax, 
    facecolor='lightblue', 
    alpha=0.3, 
    edgecolor='blue', 
    linewidth=2, 
    label='Zones éducatives (BD TOPO)'
)

gspsdt_ecoles.plot(
    ax=ax, 
    facecolor='orange', 
    alpha=0.3, 
    edgecolor='orange', 
    linewidth=2, 
    label='Potentiel Solaire des Toitures'
)

gsbab_ecoles.plot(
    ax=ax, 
    facecolor='yellow', 
    alpha=0.3, 
    edgecolor='orange', 
    linewidth=2, 
    label='Potentiel Solaire au bati'
)

# les points de l'annuaire avec zone
ecoles_avec_zone.plot(
    ax=ax,
    color='green',
    markersize=100,
    label='Écoles avec zone (Annuaire)'
)

# les points de l'annuaire sans zone
ecoles_sans_zone.plot(
    ax=ax,
    color='red',
    markersize=100,
    label='Écoles sans zone (Annuaire)'
)

cx.add_basemap(ax, crs=batiments_ecoles.crs, zoom=14)
ax.set_title(f"Zones éducatives et écoles à Saint-Denis\n"
             f"{len(batiments_ecoles)} batiments dans {len(zones_avec_ecole)} zones, \n{len(ecoles_avec_zone)} écoles avec zone\net {len(ecoles_sans_zone)} écoles sans zone", 
             pad=20)
ax.legend()

plt.show()
